### 25/11

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pickle
from astropy.io import fits
%matplotlib widget

Class example.

In [2]:
class MyFirstClass:
    internal_variable = 12345        #properties
    
    def print_hello(self):          #it is called method. self = class itself
        return 'Hello World'

In [3]:
first_class = MyFirstClass()
second_class = MyFirstClass()

print(first_class.internal_variable)
print(second_class.internal_variable)

12345
12345


In [4]:
first_class.internal_variable += 25
print(first_class.internal_variable)
print(second_class.internal_variable)


12370
12345


In [7]:
class MySecondClass:
    internal_variable = 12345

    def __init__(self, x_coord, y_coord):   #initialization of the class
        self.x_coord = x_coord
        self.test = y_coord
        print(x_coord, y_coord)

    def make_meshgrid(self, xval, yval):
        x_range = np.arange(0, xval)
        y_range = np.arange(0, yval)
        self.X, self.Y = np.meshgrid(x_range, y_range)

    def print_meshgrid(self):
        print(self.X)
        print(self.Y)
    
    def print_hello(self):
        return 'Hello World'

first_class = MySecondClass(123, 45)
second_class = MySecondClass(145, 186)

123 45
145 186


In [9]:
print(first_class.x_coord)
print(first_class.test)

first_class.make_meshgrid(10, 40)
#print(first_class.X)

first_class.print_meshgrid()

123
45
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 2  

In [ ]:
class TemporaryAperturePhotometry:
    def __init__(self):
        self.data_path = '../group08_HAT-P-12_20230214/'

        self.readout_noise = 7.1
        self.gain = 1.91

        self.bias_std = 1.3

        #loading the median bias and its associated error
        self.median_bias = pickle.load(open('../Results/median_bias.p', 'rb'))
        self.median_bias_error = pickle.load(open('../Results/median_bias_error.p', 'rb'))

        #loading the median normalized flat and its associated error
        self.median_normalized_flat = pickle.load(open('../Results/median_normalized_flat.p', 'rb'))
        self.median_normalized_flat_error = pickle.load(open('../Results/median_normalized_flat_error.p', 'rb'))

        self.science_path = self.data_path + 'science/'
        self.science_list = np.genfromtxt(self.science_path + 'science_list', dtype=str)
        self.science_size = len(self.science_list)

        ylen, xlen = np.shape(self.median_bias)
        x_axis = np.arange(0, xlen, 1.)                 #we do not save them to save memory since they are not useful for the analysis
        y_axis = np.arange(0, ylen, 1.)
        self.X, self.Y = np.meshgrid(x_axis, y_axis)

    #def provide_aperture_params(self, aperture_size, inner_radius, outer_radius) or inserting them as param in the aperture_photometry method

    def aperture_photometry(self):

        self.airmass = np.empty(self.science_size)
        self.exptime = np.empty(self.science_size)
        self.julian_date = np.empty(self.science_size)

        for i_science, science_name in enumerate(self.science_list):
            science_fits = fits.open(self.science_path + science_name)
            self.airmass[i_science] = science_fits[0].header['AIRMASS']
            self.exptime[i_science] = science_fits[0].header['EXPTIME']
            self.julian_date[i_science] = science_fits[0].header['JD']

            science_data = science_fits[0].data * self.gain
            science_corrected, science_corrected_error = self.correct_science_frame(science_data)   #we splitted the analysis because it is easier to debug

            #incorporate centroid algorithm 
            #incorporate sky bkg
            #aperture photometry

            science_fits.close()

    def correct_science_frame(self, science_data):
        science_debiased = science_data - self.median_bias
        science_corrected = science_debiased / self.median_normalized_flat

        science_debiased_error = np.sqrt(self.readout_noise**2 + science_debiased + self.median_bias_error**2)
        science_corrected_error = science_debiased * np.sqrt((science_debiased_error/science_debiased)**2 + (self.median_normalized_flat/self.median_normalized_flat_error)**2)

        return science_corrected, science_corrected_error


In [15]:
test = TemporaryAperturePhotometry()
test.aperture_photometry()
print(test.julian_date)

[2459990.562476 2459990.562637 2459990.562797 2459990.562958
 2459990.563119 2459990.56328  2459990.563441 2459990.563601
 2459990.563762 2459990.563923 2459990.564083 2459990.564244
 2459990.564406 2459990.564568 2459990.56473  2459990.564892
 2459990.565054 2459990.565217 2459990.565379 2459990.565541
 2459990.565703 2459990.565865 2459990.566027 2459990.566189
 2459990.566351 2459990.566513 2459990.566675 2459990.566837
 2459990.566999 2459990.567161 2459990.567323 2459990.567485
 2459990.567647 2459990.567809 2459990.567971 2459990.568133
 2459990.568295 2459990.568457 2459990.568619 2459990.568782
 2459990.568943 2459990.569105 2459990.569267 2459990.569429
 2459990.569591 2459990.569753 2459990.569915 2459990.570077
 2459990.570239 2459990.570401 2459990.570563 2459990.570725
 2459990.570887 2459990.571049 2459990.571211 2459990.571374
 2459990.571536 2459990.571698 2459990.57186  2459990.572022
 2459990.572184 2459990.572346 2459990.572508 2459990.57267
 2459990.572832 2459990.5

In [16]:
target_star = TemporaryAperturePhotometry()
target_star.gain

1.91